#### From time predictions to location predictions

We will try predicting the road condition in 1 meter intervals.

The issue before have been we have only used intervals, which just had the length of whatever far one could travel in 1 second. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import h5py
from scipy.interpolate import CubicSpline
from copy import deepcopy

In [9]:
parameter_dict = {
        'acc_long':     {'bstar': 198,      'rstar': 1,     'b': 198,   'r': 0.05   },
        'acc_trans':    {'bstar': 32768,    'rstar': 1,     'b': 32768, 'r': 0.04   },
        'acc_yaw':      {'bstar': 2047,     'rstar': 1,     'b': 2047,  'r': 0.1    },
        'brk_trq_elec': {'bstar': 4096,     'rstar': -1,    'b': 4098,  'r': -1     },
        'whl_trq_est':  {'bstar': 12800,    'rstar': 0.5,   'b': 12700, 'r': 1      },
        'trac_cons':    {'bstar': 80,       'rstar': 1,     'b': 79,    'r': 1      },
        'trip_cons':    {'bstar': 0,        'rstar': 0.1,   'b': 0,     'r': 1      }
    }

def convertdata(data, parameter):
    bstar = parameter['bstar']
    rstar = parameter['rstar']
    b = parameter['b']
    r = parameter['r']
    # We only convert data in the second column at idx 1 (wrt. 0-indexing), as the first column is time
    col0 = data[:,0]
    col1 = ((data[:,1]-bstar*rstar)-b)*r
    data = np.column_stack((col0, col1))
    return data


def unpack_hdf5(hdf5_file, convert: bool = False):
    with h5py.File(hdf5_file, 'r') as f:
        data = unpack_hdf5_(f, convert)
    return data


def unpack_hdf5_(group, convert: bool = False):
    data = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            data[key] = unpack_hdf5_(group[key])
        else:
            if convert and key in parameter_dict:
                data[key] = convertdata(group[key][()], parameter_dict[key])
            else:
                d = group[key][()]
                if isinstance(d, bytes):
                    data[key] = d.decode('utf-8')
                else:
                    data[key] = group[key][()]
    return data


def find_best_start_and_end_indeces_by_lonlat(trip: np.ndarray, section: np.ndarray):
    # Find the start and end indeces of the section data that are closest to the trip data
    lon_a, lat_a = trip[:,0], trip[:,1]
    lon_b, lat_b = section[:,0], section[:,1]
    
    start_index = np.argmin(np.linalg.norm(np.column_stack((lon_a, lat_a)) - np.array([lon_b[0], lat_b[0]]), axis=1))
    end_index = np.argmin(np.linalg.norm(np.column_stack((lon_a, lat_a)) - np.array([lon_b[-1], lat_b[-1]]), axis=1))

    return start_index, end_index


def time_to_drive_X_meters(speed_kmh, distance_m=10):
    # Convert speed from km/h to m/s
    speed_ms = speed_kmh * (1000 / 3600)

    # Calculate time in seconds
    time_seconds = distance_m / speed_ms
    
    return time_seconds

In [26]:
autopi_hh = unpack_hdf5('../data/raw/AutoPi_CAN/platoon_CPH1_HH.hdf5')
gm_data = autopi_hh['GM']['16006']['pass_1'] # TODO choose a one true route. 
p79_hh = pd.read_csv('../data/raw/ref_data/cph1_zp_hh.csv', sep=';', encoding='unicode_escape')
p79_vh = pd.read_csv('../data/raw/ref_data/cph1_zp_vh.csv', sep=';', encoding='unicode_escape')

In [24]:
# HH 
# Extract every 10th item starting from idx[0] to idx[1]+1, to get one location for each meter
lon_zp_hh = p79_hh['Lon']
lat_zp_hh = p79_hh['Lat']
idx_hh = find_best_start_and_end_indeces_by_lonlat(p79_hh[['Lat', 'Lon']].to_numpy(), gm_data['gps'][:,1:])
loc_hh_lon = lon_zp_hh[idx_hh[0]:idx_hh[1]+1:10]
loc_hh_lat = lat_zp_hh[idx_hh[0]:idx_hh[1]+1:10]

# Combine lon and lat into a list of lists
loc_hh = [[lon, lat] for lon, lat in zip(loc_hh_lon, loc_hh_lat)]

# For demonstration, let's print the first few pairs to verify
print(loc_hh[:5])

[[12.53019946, 55.7111808], [12.53018721, 55.71117492], [12.53017498, 55.71116905], [12.53016273, 55.71116317], [12.53015047, 55.7111573]]


In [27]:
# VH
# Extract every 10th item starting from idx[0] to idx[1]+1, to get one location for each meter
lon_zp_vh = p79_vh['Lon']
lat_zp_vh = p79_vh['Lat']
idx_vh = find_best_start_and_end_indeces_by_lonlat(p79_vh[['Lat', 'Lon']].to_numpy(), gm_data['gps'][:,1:])
loc_vh_lon = lon_zp_vh[idx_hh[0]:idx_vh[1]+1:10]
loc_vh_lat = lat_zp_vh[idx_hh[0]:idx_vh[1]+1:10]

# Combine lon and lat into a list of lists
loc_vh = [[lon, lat] for lon, lat in zip(loc_hh_lon, loc_hh_lat)]

# For demonstration, let's print the first few pairs to verify
print(loc_vh[:5])

[[12.53019946, 55.7111808], [12.53018721, 55.71117492], [12.53017498, 55.71116905], [12.53016273, 55.71116317], [12.53015047, 55.7111573]]


In [29]:
# Create a function that map each location to a 1 second time window.
loc_cars_hh = {}
loc_cars_vh = {}

cars = ["car_1", "car_2", "car_3"] 

for AIO_lan_lon in [loc_cars_hh, loc_cars_vh]:
    for car in cars:
        AIO_lan_lon[car] = []
        print(AIO_lan_lon)
        # go trough each location and map it to a 1 second time window
        for lan, lon in AIO_lan_lon:
            print(lan)
            break
            # found_window = 0 #TODO: find the window that is closest to the location
            # AIO_lan_lon[car].apend(found_window)
        

{'car_1': []}


ValueError: too many values to unpack (expected 2)

#### After finding an mapping each AOI to a time window, we will predict the KPI of the AOI

In [7]:
# Use the speed to estimate the amount of sample points there are in the area of interest (AOI)
# The AOI is thought to be defined by an X meter long segment
A_speed = 50
B_speed = 30
C_speed = 40

sample_freq = 250  # Sample frequency in Hz
A_samples = time_to_drive_X_meters(A_speed) * sample_freq
B_samples = time_to_drive_X_meters(B_speed) * sample_freq
C_samples = time_to_drive_X_meters(C_speed) * sample_freq
total_samples = A_samples + B_samples + C_samples

A_weight = A_samples / total_samples
B_weight = B_samples / total_samples
C_weight = C_samples / total_samples


In [8]:
# We will now localise the point of interest in the time domains.
# It does not work to just take the first and last lon. and lat. and not the median either. 
# We need to be able to find the location fast, right now we will just 

# Should they all have a dictionary?
A_t1_data = [[1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10]]
A_t1_pred = 5
